# シンプルな特徴量作成

In [1]:
# General imports
import numpy as np
import pandas as pd
import os, sys, gc, time, warnings, pickle, psutil, random

from math import ceil

from sklearn.preprocessing import LabelEncoder

warnings.filterwarnings('ignore')

In [2]:
## Simple "Memory profilers" to see memory usage
# CPUのmemoryデータを見る
def get_memory_usage():
    return np.round(psutil.Process(os.getpid()).memory_info()[0]/2.**30, 2) 


def sizeof_fmt(num, suffix='B'):
    for unit in ['','Ki','Mi','Gi','Ti','Pi','Ei','Zi']:
        if abs(num) < 1024.0:
            return "%3.1f%s%s" % (num, unit, suffix)
        num /= 1024.0
    return "%.1f%s%s" % (num, 'Yi', suffix)

In [3]:
## Memory Reducer
# 省メモリ用の関数
# :df pandas dataframe to reduce size             # type: pd.DataFrame()
# :verbose                                        # type: bool
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                       df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df

In [4]:
## Merging by concat to not lose dtypes
# 左結合を行う関数
def merge_by_concat(df1, df2, merge_on):
    merged_gf = df1[merge_on]
    merged_gf = merged_gf.merge(df2, on=merge_on, how='left')
    new_columns = [col for col in list(merged_gf) if col not in merge_on]
    df1 = pd.concat([df1, merged_gf[new_columns]], axis=1)
    return df1

In [5]:
########################### Vars
#################################################################################
TARGET = 'sales'         # Our main target
START_TRAIN = 29
END_TRAIN = 1941         # Last day in train set
MAIN_INDEX = ['id','d']  # We can identify item by these columns

* train_dfのデータをevaluationを含めて再作成する  
evaluationデータを加算するとメモリがオーバーしてしまうのでトレーニングデータを29日目以降にしてみる

In [6]:
########################### Load Data
#################################################################################
print('Load Main Data')

# Here are reafing all our data 
# without any limitations and dtype modification
train_df = pd.read_csv('../input/m5-forecasting-accuracy/sales_train_evaluation.csv')
prices_df = pd.read_csv('../input/m5-forecasting-accuracy/sell_prices.csv')
calendar_df = pd.read_csv('../input/m5-forecasting-accuracy/calendar.csv')

Load Main Data


In [7]:
train_temp = train_df.iloc[:, :6]
train_df = pd.concat([train_temp, train_df.iloc[:, 5 + START_TRAIN:]], axis=1)
del train_temp

In [8]:
# startとendを除いたNBA Finalsの日程
nba_finals_dates = [
    "2011-06-02", "2011-06-05", "2011-06-07",
    "2011-06-09", 
    "2012-06-14", "2012-06-17", "2012-06-19",
    "2013-06-09", "2013-06-11", "2013-06-13",
    "2013-06-16", "2013-06-18", 
    "2014-06-08", "2014-06-10", "2014-06-12",
    "2015-06-07", "2015-06-09", "2015-06-11",
    "2015-06-14", 
    "2016-06-05", "2016-06-08", "2016-06-10",
    "2016-06-13", "2016-06-16"
]

nba_index=pd.DataFrame({})
for nba in (nba_finals_dates):
    nba_index = nba_index.append(calendar_df[calendar_df['date']==nba])

In [9]:
nba_index_l = nba_index.index
for l in (nba_index_l):
    if calendar_df.at[l, 'event_type_1'] == float('Nan'):
        calendar_df.at[l, 'event_type_1'] = 'Sporting'
        calendar_df.at[l, 'event_name_1'] = 'NBA'
    else:
        calendar_df.at[l, 'event_type_2'] = 'Sporting'
        calendar_df.at[l, 'event_name_2'] = 'NBA'

In [10]:
del nba_index_l, nba_index, nba_finals_dates

* event_type:SaleとしてBlack Fridayを追加した。

In [11]:
# ブラックフライデーと正月・クリスマスは特徴量を作成しておくことが推奨されている。
# 正月・クリスマスに関しては既にevent-type-1に追加されているのでブラックフライデーを考えたい。
# ブラックフライデー：11月の最終金曜日
# ブラックフライデーの前日、前々日程度は特徴量として持っておく。
black_friday = [
    "2011-11-25", "2012-11-23", "2013-11-29", "2014-11-28", "2015-11-27" 
]
black_friday_before = [
    "2011-11-24", "2012-11-22", "2013-11-28", "2014-11-27", "2015-11-26"
]
black_friday_after = [
    "2011-11-26", "2012-11-24", "2013-11-30", "2014-11-29", "2015-11-28"
]

black_index=pd.DataFrame({})
for black in (black_friday):
    black_index = black_index.append(calendar_df[calendar_df['date']==black])

In [12]:
black_index_l = black_index.index
for l in (black_index_l):
    if calendar_df.at[l, 'event_type_1'] == float('Nan'):
        calendar_df.at[l, 'event_type_1'] = 'Sale'
        calendar_df.at[l, 'event_name_1'] = 'Black Friday'
    else:
        calendar_df.at[l, 'event_type_2'] = 'Sale'
        calendar_df.at[l, 'event_name_2'] = 'Black Friday'

In [13]:
del black_index, black_friday, black_index_l

In [14]:
# black_friday_beforeの追加
black_index=pd.DataFrame({})
for black in (black_friday_before):
    black_index = black_index.append(calendar_df[calendar_df['date']==black])
black_index_l = black_index.index
for l in (black_index_l):
    if calendar_df.at[l, 'event_type_1'] == float('Nan'):
        calendar_df.at[l, 'event_type_1'] = 'Sale'
        calendar_df.at[l, 'event_name_1'] = 'Black Friday Before'
    else:
        calendar_df.at[l, 'event_type_2'] = 'Sale'
        calendar_df.at[l, 'event_name_2'] = 'Black Friday Before'
del black_index, black_friday_before, black_index_l

In [15]:
# black_friday_afterの追加
black_index=pd.DataFrame({})
for black in (black_friday_after):
    black_index = black_index.append(calendar_df[calendar_df['date']==black])
black_index_l = black_index.index
for l in (black_index_l):
    if calendar_df.at[l, 'event_type_1'] == float('Nan'):
        calendar_df.at[l, 'event_type_1'] = 'Sale'
        calendar_df.at[l, 'event_name_1'] = 'Black Friday Before'
    else:
        calendar_df.at[l, 'event_type_2'] = 'Sale'
        calendar_df.at[l, 'event_name_2'] = 'Black Friday Before'
del black_index, black_friday_after, black_index_l

* クリスマスは全店休業日となっている。トレーニングデータから除外してしまってもいいかもしれない。
* 祝日データを追加したい。

In [16]:
########################### Make Grid
#################################################################################
print('Create Grid')

# We can tranform horizontal representation 
# to vertical "view"
# Our "index" will be 'id','item_id','dept_id','cat_id','store_id','state_id'
# and labels are 'd_' coulmns

'''
元データではitem_idやdept_idなどのidデータが変数名として扱われている。
これをidデータとして扱うため、pd.meltでデータ変形を行う。
→以下のようなデータフレームへの変形となる。
	id	item_id	dept_id	cat_id	store_id	state_id	d	sales
0	HOBBIES_1_001_CA_1_validation	HOBBIES_1_001	HOBBIES_1	HOBBIES	CA_1	CA	d_1	0
1	HOBBIES_1_002_CA_1_validation	HOBBIES_1_002	HOBBIES_1	HOBBIES	CA_1	CA	d_1	0
2	HOBBIES_1_003_CA_1_validation	HOBBIES_1_003	HOBBIES_1	HOBBIES	CA_1	CA	d_1	0
3	HOBBIES_1_004_CA_1_validation	HOBBIES_1_004	HOBBIES_1	HOBBIES	CA_1	CA	d_1	0
4	HOBBIES_1_005_CA_1_validation	HOBBIES_1_005	HOBBIES_1	HOBBIES	CA_1	CA	d_1	0

目的変数を一意に定める説明変数の作成を行う形
'''
index_columns = ['id','item_id','dept_id','cat_id','store_id','state_id']
grid_df = pd.melt(train_df, 
                  id_vars = index_columns, 
                  var_name = 'd', 
                  value_name = TARGET)

# If we look on train_df we se that 
# we don't have a lot of traning rows
# but each day can provide more train data
print('Train rows:', len(train_df), len(grid_df))

# To be able to make predictions
# we need to add "test set" to our grid
# test用のデータ作成。add_gridは1914日目から28日間のデータを格納する。
add_grid = pd.DataFrame()
for i in range(1,29):
    temp_df = train_df[index_columns]
    temp_df = temp_df.drop_duplicates()
    temp_df['d'] = 'd_'+ str(END_TRAIN+i)
    temp_df[TARGET] = np.nan
    add_grid = pd.concat([add_grid,temp_df])

grid_df = pd.concat([grid_df,add_grid])# テスト用データの追加
grid_df = grid_df.reset_index(drop=True)# indexを連番に振り直す

# メモリ節約
# Remove some temoprary DFs
del temp_df, add_grid

# We will not need original train_df
# anymore and can remove it
del train_df

# You don't have to use df = df construction
# you can use inplace=True instead.
# like this
# grid_df.reset_index(drop=True, inplace=True)

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# We can free some memory 
# by converting "strings" to categorical
# it will not affect merging and 
# we will not lose any valuable data
# grid_dfのdtypesを単純なstrからcategoryに変換することでメモリ削減を図っている（？）
for col in index_columns:
    grid_df[col] = grid_df[col].astype('category')

# Let's check again memory usage
# メモリの食い方を確認してみるとメモリ削減関数を使用したことで約1/3程度のメモリでgrid_dfが形成されていることがわかる。
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

Create Grid
Train rows: 30490 58327370
    Original grid_df:   3.5GiB
     Reduced grid_df:   1.3GiB


## 特徴量を増やすとしたらここらあたりから 
### まずは売上があがったタイミングをデータに反映するpart1

In [17]:
########################### Product Release date
#################################################################################
print('Release week')

# It seems that leadings zero values
# in each train_df item row
# are not real 0 sales but mean
# absence for the item in the store
# we can safe some memory by removing
# such zeros

# Prices are set by week
# so it we will have not very accurate release week 
# 売上があがった初週を取得
release_df = prices_df.groupby(['store_id','item_id'])['wm_yr_wk'].agg(['min']).reset_index()# 降順で並べ直す
release_df.columns = ['store_id','item_id','release']

# Now we can merge release_df
# 売上が上がった初週のデータをgrid_dfにマージ
grid_df = merge_by_concat(grid_df, release_df, ['store_id','item_id'])
del release_df

# We want to remove some "zeros" rows
# from grid_df 
# to do it we need wm_yr_wk column
# let's merge partly calendar_df to have it
grid_df = merge_by_concat(grid_df, calendar_df[['wm_yr_wk','d']], ['d'])
                      
# Now we can cutoff some rows 
# and safe memory 
# 該当品目の最初の売上が上がるまでの週は不要なデータとして削除を行う。
grid_df = grid_df[grid_df['wm_yr_wk']>=grid_df['release']]
grid_df = grid_df.reset_index(drop=True)

# Let's check our memory usage
print("{:>20}: {:>8}".format('Original grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

# Should we keep release week 
# as one of the features?
# Only good CV can give the answer.
# Let's minify the release values.
# Min transformation will not help here 
# as int16 -> Integer (-32768 to 32767)
# and our grid_df['release'].max() serves for int16
# but we have have an idea how to transform 
# other columns in case we will need it
# 売上初週のデータを特徴量として保持すべきかはCVによって決定すべき内容。現時点では保持しておく。
grid_df['release'] = grid_df['release'] - grid_df['release'].min()
grid_df['release'] = grid_df['release'].astype(np.int16)

# Let's check again memory usage
print("{:>20}: {:>8}".format('Reduced grid_df',sizeof_fmt(grid_df.memory_usage(index=True).sum())))

Release week
    Original grid_df:   1.8GiB
     Reduced grid_df:   1.5GiB


In [18]:
########################### Save part 1
#################################################################################
print('Save Part 1')

# We have our BASE grid ready
# and can save it as pickle file
# for future use (model training)
grid_df.to_pickle('grid_part_1.pkl')

print('Size:', grid_df.shape)

Save Part 1
Size: (47397822, 10)


### part2：priceデータの反映

In [19]:
########################### Prices ここの特徴量を増やすとメモリがオーバーしてしまうのでここはデフォルトのままで進める
#################################################################################
print('Prices')

# We can do some basic aggregations
# 店舗・品目別のmax, min, std, meanのpricesをデータに追加する。
prices_df['price_max'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('max')
prices_df['price_min'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('min')
prices_df['price_std'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('std')
prices_df['price_mean'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('mean')

# and do price normalization (min/max scaling)
# さらにmin-max正規化を行った行の追加
prices_df['price_norm'] = prices_df['sell_price']/prices_df['price_max']

# Some items are can be inflation dependent
# and some items are very "stable"
# pricesの変動がどれだけ起きたかを行に追加する。
prices_df['price_nunique'] = prices_df.groupby(['store_id','item_id'])['sell_price'].transform('nunique')
prices_df['item_nunique'] = prices_df.groupby(['store_id','sell_price'])['item_id'].transform('nunique')

# I would like some "rolling" aggregations
# but would like months and years as "window"
# 周期的なデータ拡張を行うため、年次、月次のデータをcalendar_dfから追加する。
calendar_prices = calendar_df[['wm_yr_wk','month','year']]
calendar_prices = calendar_prices.drop_duplicates(subset=['wm_yr_wk'])
prices_df = prices_df.merge(calendar_prices[['wm_yr_wk','month','year']], on=['wm_yr_wk'], how='left')
del calendar_prices

# Now we can add price "momentum" (some sort of)
# Shifted by week 
# by month mean
# by year mean
# 年次ごと、月次ごとのprices集計を行い、この平均とその時点でのpriceを割ることでモーメンタムの作成を行う
prices_df['price_momentum'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id'])['sell_price'].transform(lambda x: x.shift(1))
prices_df['price_momentum_m'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','month'])['sell_price'].transform('mean')
prices_df['price_momentum_y'] = prices_df['sell_price']/prices_df.groupby(['store_id','item_id','year'])['sell_price'].transform('mean')

del prices_df['month'], prices_df['year']

Prices


In [20]:
########################### Merge prices and save part 2
#################################################################################
print('Merge prices and save part 2')

# Merge Prices
original_columns = list(grid_df)
grid_df = grid_df.merge(prices_df, on=['store_id','item_id','wm_yr_wk'], how='left')
keep_columns = [col for col in list(grid_df) if col not in original_columns]
grid_df = grid_df[MAIN_INDEX+keep_columns]
grid_df = reduce_mem_usage(grid_df)

# Safe part 2
grid_df.to_pickle('grid_part_2.pkl')
print('Size:', grid_df.shape)

# We don't need prices_df anymore
del prices_df

# We can remove new columns
# or just load part_1
grid_df = pd.read_pickle('grid_part_1.pkl')

Merge prices and save part 2
Mem. usage decreased to 1809.57 Mb (62.2% reduction)
Size: (47397822, 13)


### part3：calendarデータを使用して特徴量を作成

In [21]:
########################### Merge calendar
#################################################################################
grid_df = grid_df[MAIN_INDEX]

# Merge calendar partly
icols = ['date',
         'd',
         'event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']

grid_df = grid_df.merge(calendar_df[icols], on=['d'], how='left')

# Minify data
# 'snap_' columns we can convert to bool or int8
icols = ['event_name_1',
         'event_type_1',
         'event_name_2',
         'event_type_2',
         'snap_CA',
         'snap_TX',
         'snap_WI']
for col in icols:
    grid_df[col] = grid_df[col].astype('category')

# Convert to DateTime
grid_df['date'] = pd.to_datetime(grid_df['date'])

# Make some features from date
# pandas.dtメソッドによって日付データを日次、週次、月次でデータ変換を行い特徴量として追加する。
grid_df['tm_d'] = grid_df['date'].dt.day.astype(np.int8)
grid_df['tm_w'] = grid_df['date'].dt.week.astype(np.int8)
grid_df['tm_m'] = grid_df['date'].dt.month.astype(np.int8)
grid_df['tm_y'] = grid_df['date'].dt.year
grid_df['tm_y'] = (grid_df['tm_y'] - grid_df['tm_y'].min()).astype(np.int8)
grid_df['tm_wm'] = grid_df['tm_d'].apply(lambda x: ceil(x/7)).astype(np.int8)

grid_df['tm_dw'] = grid_df['date'].dt.dayofweek.astype(np.int8)
grid_df['tm_w_end'] = (grid_df['tm_dw']>=5).astype(np.int8)

# Remove date
del grid_df['date']

In [22]:
########################### Save part 3 (Dates)
#################################################################################
print('Save part 3')

# Safe part 3
grid_df.to_pickle('grid_part_3.pkl')
print('Size:', grid_df.shape)

# We don't need calendar_df anymore
del calendar_df
del grid_df

Save part 3
Size: (47397822, 16)


In [23]:
########################### Some additional cleaning
#################################################################################

## Part 1
# Convert 'd' to int
grid_df = pd.read_pickle('grid_part_1.pkl')
grid_df['d'] = grid_df['d'].apply(lambda x: x[2:]).astype(np.int16)

# Remove 'wm_yr_wk'
# as test values are not in train set
del grid_df['wm_yr_wk']
grid_df.to_pickle('grid_part_1.pkl')

del grid_df

In [24]:
'''
########################### Summary
#################################################################################

# Now we have 3 sets of features
# grid_part_1~3の3種類のinputデータを結合することによってfull gridデータの作成が完了する。
# ただし、full girdデータをそのままトレーニングに使用するとデータが巨大すぎる。
grid_df = pd.concat([pd.read_pickle('grid_part_1.pkl'),
                     pd.read_pickle('grid_part_2.pkl').iloc[:,2:],
                     pd.read_pickle('grid_part_3.pkl').iloc[:,2:]],
                     axis=1)
                     
# Let's check again memory usage
print("{:>20}: {:>8}".format('Full Grid',sizeof_fmt(grid_df.memory_usage(index=True).sum())))
print('Size:', grid_df.shape)

# 2.5GiB + is is still too big to train our model
# (on kaggle with its memory limits)
# and we don't have lag features yet
# But what if we can train by state_id or shop_id?
# full gridデータを使用するとデータが巨大すぎるので州ごと、もしくは店舗ごとにトレーニングを行うことでメモリオーバーを防げるのではないかと考えている。
state_id = 'CA'
grid_df = grid_df[grid_df['state_id']==state_id]
print("{:>20}: {:>8}".format('Full Grid',sizeof_fmt(grid_df.memory_usage(index=True).sum())))
#           Full Grid:   1.2GiB

store_id = 'CA_1'
grid_df = grid_df[grid_df['store_id']==store_id]
print("{:>20}: {:>8}".format('Full Grid',sizeof_fmt(grid_df.memory_usage(index=True).sum())))
#           Full Grid: 321.2MiB

# Seems its good enough now
# In other kernel we will talk about LAGS features
# Thank you.
'''

'\n########################### Summary\n#################################################################################\n\n# Now we have 3 sets of features\n# grid_part_1~3の3種類のinputデータを結合することによってfull gridデータの作成が完了する。\n# ただし、full girdデータをそのままトレーニングに使用するとデータが巨大すぎる。\ngrid_df = pd.concat([pd.read_pickle(\'grid_part_1.pkl\'),\n                     pd.read_pickle(\'grid_part_2.pkl\').iloc[:,2:],\n                     pd.read_pickle(\'grid_part_3.pkl\').iloc[:,2:]],\n                     axis=1)\n                     \n# Let\'s check again memory usage\nprint("{:>20}: {:>8}".format(\'Full Grid\',sizeof_fmt(grid_df.memory_usage(index=True).sum())))\nprint(\'Size:\', grid_df.shape)\n\n# 2.5GiB + is is still too big to train our model\n# (on kaggle with its memory limits)\n# and we don\'t have lag features yet\n# But what if we can train by state_id or shop_id?\n# full gridデータを使用するとデータが巨大すぎるので州ごと、もしくは店舗ごとにトレーニングを行うことでメモリオーバーを防げるのではないかと考えている。\nstate_id = \'CA\'\ngrid_df = grid_df[grid_df[\'sta